In [ ]:
#from google.colab import drive
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from scipy.io import loadmat
from shutil import copyfile
from time import time
import csv 
import cv2
import json
import numpy as np
import os 
import pandas as pd
import random
import tensorflow as tf
import tensorflow_addons as tfa
import zipfile

In [ ]:
# ----- Loading the dataset -----

#drive.mount('/content/gdrive', force_remount=True)
t = time()

# Zipped Files
for file_name in ["Annotations", "emotic"]:
  if os.path.exists(f'{file_name}/'):
    print(f'{file_name} already loaded')
  else:
    print(f'Loading of {file_name}.zip ...')
    if os.path.exists(f'gdrive/MyDrive/FDL_Project/{file_name}.zip'):
      # Copy from GDrive to Colab VM
      copyfile(f'gdrive/MyDrive/FDL_Project/{file_name}.zip', f'{file_name}.zip')
      # Extract files
      zip = zipfile.ZipFile(f'{file_name}.zip')
      zip.extractall()
      zip.close()
    else:
      print(f'{file_name} not present in Gdrive')

# Normal files
for file_name in ["Annotations.json"]:
  if os.path.exists(f'{file_name}'):
    print(f'{file_name} already loaded')
  else:
    print(f'Loading of {file_name}...')
    if os.path.exists(f'gdrive/MyDrive/FDL_Project/{file_name}'):
      # Copy from GDrive to Colab VM
      copyfile(f'gdrive/MyDrive/FDL_Project/{file_name}', f'{file_name}')
    else:
      print(f'{file_name} not present in Gdrive')
print(f'File transfer completed in {round(time() - t, 2)} seconds')

In [ ]:
# ----- Parsing of the annotations -----

class dataset_train:
    def __init__(self, filename, folder, image_size, person):
        self.filename = filename
        self.folder = folder
        self.im_size = []
        self.bbox = []
        self.cat = []
        self.cont = []
        self.gender = person[3][0]
        self.age = person[4][0]
        self.cat_annotators = 0
        self.cont_annotators = 0
        self.set_imsize(image_size)
        self.set_bbox(person[0])
        self.set_cat(person[1])
        self.set_cont(person[2])
        self.check_cont()

    def set_imsize(self, image_size):
        image_size = np.array(image_size).flatten().tolist()[0]
        row = np.array(image_size[0]).flatten().tolist()[0]
        col = np.array(image_size[1]).flatten().tolist()[0]
        self.im_size.append(row)
        self.im_size.append(col)

    def validate_bbox(self, bbox):
        x1, y1, x2, y2 = bbox
        x1 = min(self.im_size[0], max(0, x1))
        x2 = min(self.im_size[0], max(0, x2))
        y1 = min(self.im_size[1], max(0, y1))
        y2 = min(self.im_size[1], max(0, y2))
        return [int(x1), int(y1), int(x2), int(y2)]

    def set_bbox(self, person_bbox):
        self.bbox = self.validate_bbox(np.array(person_bbox).flatten().tolist())

    def set_cat(self, person_cat):
        cat = np.array(person_cat).flatten().tolist()
        cat = np.array(cat[0]).flatten().tolist()
        self.cat = [np.array(c).flatten().tolist()[0] for c in cat]
        self.cat_annotators = 1

    def set_cont(self, person_cont):
        cont = np.array(person_cont).flatten().tolist()[0]
        self.cont = [np.array(c).flatten().tolist()[0] for c in cont]
        self.cont_annotators = 1

    def check_cont(self):
        for c in self.cont:
            if np.isnan(c):
                self.cont_annotators = 0
                break

class dataset_test:
    def __init__(self, filename, folder, image_size, person):
        self.filename = filename
        self.folder = folder
        self.im_size = []
        self.bbox = []
        self.cat = []
        self.cat_annotators = 0
        self.comb_cat = []
        self.cont_annotators = 0
        self.cont = []
        self.comb_cont = []
        self.gender = person[5][0]
        self.age = person[6][0]

        self.set_imsize(image_size)
        self.set_bbox(person[0])
        self.set_cat(person[1])
        self.set_comb_cat(person[2])
        self.set_cont(person[3])
        self.set_comb_cont(person[4])
        self.check_cont()

    def set_imsize(self, image_size):
        image_size = np.array(image_size).flatten().tolist()[0]
        row = np.array(image_size[0]).flatten().tolist()[0]
        col = np.array(image_size[1]).flatten().tolist()[0]
        self.im_size.append(row)
        self.im_size.append(col)

    def validate_bbox(self, bbox):
        x1, y1, x2, y2 = bbox
        x1 = min(self.im_size[0], max(0, x1))
        x2 = min(self.im_size[0], max(0, x2))
        y1 = min(self.im_size[1], max(0, y1))
        y2 = min(self.im_size[1], max(0, y2))
        return [int(x1), int(y1), int(x2), int(y2)]

    def set_bbox(self, person_bbox):
        self.bbox = self.validate_bbox(np.array(person_bbox).flatten().tolist())

    def set_cat(self, person_cat):
        self.cat_annotators = len(person_cat[0])
        for ann in range(self.cat_annotators):
            ann_cat = person_cat[0][ann]
            ann_cat = np.array(ann_cat).flatten().tolist()
            ann_cat = np.array(ann_cat[0]).flatten().tolist()
            ann_cat = [np.array(c).flatten().tolist()[0] for c in ann_cat]
            self.cat.append(ann_cat)

    def set_comb_cat(self, person_comb_cat):
        if self.cat_annotators != 0:
            self.comb_cat = [np.array(c).flatten().tolist()[0] for c in person_comb_cat[0]]
        else:
            self.comb_cat = []

    def set_comb_cont(self, person_comb_cont):
        if self.cont_annotators != 0:
            comb_cont = [np.array(c).flatten().tolist()[0] for c in person_comb_cont[0]]
            self.comb_cont = [np.array(c).flatten().tolist()[0] for c in comb_cont[0]]
        else:
            self.comb_cont = []

    def set_cont(self, person_cont):
        self.cont_annotators = len(person_cont[0])
        for ann in range(self.cont_annotators):
            ann_cont = person_cont[0][ann]
            ann_cont = np.array(ann_cont).flatten().tolist()
            ann_cont = np.array(ann_cont[0]).flatten().tolist()
            ann_cont = [np.array(c).flatten().tolist()[0] for c in ann_cont]
            self.cont.append(ann_cont)

    def check_cont(self):
        for c in self.comb_cont:
            if np.isnan(c):
                self.cont_annotators = 0
                break

save_path = 'emotic/parsed'
if not os.path.exists(save_path):
  os.makedirs(save_path)

cat = ['Affection', 'Anger', 'Annoyance', 'Anticipation', 'Aversion', 'Confidence', 'Disapproval', 'Disconnection',
    'Disquietment', 'Doubt/Confusion', 'Embarrassment', 'Engagement', 'Esteem', 'Excitement', 'Fatigue', 'Fear',
    'Happiness', 'Pain', 'Peace', 'Pleasure', 'Sadness', 'Sensitivity', 'Suffering', 'Surprise', 'Sympathy', 'Yearning']

cat2ind = {}
ind2cat = {}

for idx, emotion in enumerate(cat):
    cat2ind[emotion] = idx
    ind2cat[idx] = emotion

print ('Loading annotations...')
t = time()
if os.path.exists('Annotations.json'):
    print ('Annotations already loaded')
else:
  mat = loadmat('Annotations/Annotations.mat')
  labels = ['train', 'val', 'test']
  for label in labels:
    if os.path.exists(f'emotic/parsed/{label}.csv'):
      print (f'Label {label} already loaded')
    else:
      data_mat = mat[label]
      print (f'Loading label {label}')
      data_set = list()
      to_break = 0
      path_not_exist = 0
      cat_cont_zero = 0
      idx = 0
      for ex_idx, ex in enumerate(data_mat[0]):
        nop = len(ex[4][0])
        for person in range(nop):
          if label == 'train':
            et = dataset_train(ex[0][0],ex[1][0],ex[2],ex[4][0][person])
          else:
            et = dataset_test(ex[0][0],ex[1][0],ex[2],ex[4][0][person])
          try:
            image_path = os.path.join('emotic/',et.folder,et.filename)
            if not os.path.exists(image_path):
              path_not_exist += 1
              print ('path not existing', ex_idx, image_path)
              continue
            else:
              context = cv2.cvtColor(cv2.imread(image_path),cv2.COLOR_BGR2RGB)
              body = context[et.bbox[1]:et.bbox[3],et.bbox[0]:et.bbox[2]].copy()
              context_cv = cv2.resize(context, (224,224))
              body_cv = cv2.resize(body, (128,128))
          except Exception as e:
            to_break += 1
            continue
          if (et.cat_annotators == 0 or et.cont_annotators == 0):
            cat_cont_zero += 1
            continue
          data_set.append(et)
          idx += 1  
          if idx % 1000 == 0:
            print (" Preprocessing data. Index = ", idx)
      print (to_break, path_not_exist, cat_cont_zero)
      
      csv_path = os.path.join(save_path, "%s.csv" %(label))
      with open(csv_path, 'w') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',', dialect='excel')
        row = ['Index', 'Folder', 'Filename', 'Image Size', 'BBox', 'Categorical_Labels', 'Continuous_Labels', 'Gender', 'Age']
        filewriter.writerow(row)
        for idx, ex in enumerate(data_set):
            if label == 'train':
                row = [idx, ex.folder, ex.filename, ex.im_size, ex.bbox, ex.cat, ex.cont, ex.gender, ex.age]
            else:
                row = [idx, ex.folder, ex.filename, ex.im_size, ex.bbox, ex.comb_cat, ex.comb_cont, ex.gender, ex.age]
            filewriter.writerow(row)
      print ('wrote file ', csv_path)
      print ('completed generating %s data files' %(label))

print(f'Annotation loading completed in {round(time() - t, 2)} seconds')

In [ ]:
# ----- Print Images with Annotations -----

def parse_categories(labels):
    categories = [0 for i in range(26)]
    for x in labels:
        categories[cat2ind[x]]=1
    return categories

def parse_index_from_cat(labels):
    categories = []
    for idx,i in enumerate(labels):
        if(i)>0.1:
            categories.append(ind2cat[idx])
    return categories

def printImage(bboxes, labels, img, scaled=True, resnet=False, size = [], label_mode=False):

  # Plot figure
  fig = plt.figure()
  
  # add axes to the image
  ax = fig.add_axes([0, 0, 1, 1])

  # read and plot the image
  if isinstance(img, str):
    img = plt.imread(img) 

  height = img.shape[0]
  width = img.shape[1]
  if size != []:
    height = size[0]
    width = size[1]
  
  plt.imshow(img)
  for i in range(len(bboxes)):
    #print(bboxes[i])
    bbox = bboxes[i]
    if isinstance(bbox, str):
      bbox = [float(x) for x in bbox[1:-1].split(',')]

    ymin, xmin, ymax, xmax, w, h = [0,0,0,0,0,0]

    if scaled:
      if resnet:
        bbox[1] *= width
        bbox[2] *= height
        bbox[3] *= width
        bbox[0] *= height
        ymin, xmin, ymax, xmax = bbox
        w = xmax - xmin
        h = ymax - ymin
      else:
        bbox[0] *= width
        bbox[1] *= height
        bbox[2] *= width
        bbox[3] *= height
        xmin, ymin, xmax, ymax = bbox
        w = xmax - xmin
        h = ymax - ymin
    else:
      xmin, ymin, xmax, ymax = bbox
      w = xmax - xmin
      h = ymax - ymin

    # add bounding boxes to the image
    box = Rectangle(
        (xmin, ymin), w, h, edgecolor="red", facecolor="none"
    )

    ax.add_patch(box)

    rx, ry = box.get_xy()
    cx = rx + box.get_width()/2.0
    cy = ry + box.get_height()/8.0

    box_label = labels[i]
    if label_mode:
      box_label = parse_index_from_cat(labels[i])

    l = ax.annotate(
      box_label,
      (cx, cy),
      fontsize=8,
      fontweight="bold",
      color="white",
      ha='center',
      va='center'
    )
    l.set_bbox(
      dict(facecolor='red', alpha=0.5, edgecolor='red')
    )

In [ ]:
# ----- Annotation Parsing Pt.2 -----

MAX_BOX = 1

annotation = {
    "train": [],
    "val": [],
    "test": [],
}

limit = {
  "val": 1300,
  "train": 5000,
  "test": 1300
}

if os.path.exists('Annotations.json'):
    with open('Annotations.json', 'r') as openfile:
      annotation = json.load(openfile)
    print('Annotations already loaded')
    #for x in [random.randint(0, len(annotation[label])) for x in range(1)]:
      #printImage([annotation[label][x]["bbox"]], annotation[label][x]["label"], annotation[label][x]["path"]) 
else:
  for label in ["val", "train", "test"]:

    # Group dataset by image path
    image_dataframe = pd.read_csv(f'emotic/parsed/{label}.csv')
    image_dataframe["image_path"] = image_dataframe.apply(lambda x: os.path.join('emotic',x['Folder'], x['Filename']), axis=1)
    categories = image_dataframe.groupby("image_path")['Categorical_Labels'].apply(list)
    continuous_labels = image_dataframe.groupby("image_path")['Continuous_Labels'].apply(list)
    image_dataframe = pd.DataFrame(image_dataframe.groupby("image_path")['BBox'].apply(list))
    image_dataframe = image_dataframe.rename(columns={"BBox":"bbox"})
    image_dataframe['label'] = categories
    image_dataframe['continuous_labels'] = continuous_labels
    #display(image_dataframe)  

    # Parse to python array
    count = 0
    for index, row in  image_dataframe.iterrows():
      if len(row["bbox"]) <= MAX_BOX and len(row["label"]) <= MAX_BOX and len(row["continuous_labels"]) <= MAX_BOX:
        img = plt.imread(row.name)
        height = img.shape[0]
        width = img.shape[1]
        new_bboxes = []
        new_cont_labels = []
        new_labels = []
        for index,element in enumerate(row["bbox"]):
            df_bbox = element
            bbox = [float(x) for x in df_bbox[1:-1].split(',')]
            bbox[0] /= width
            bbox[1] /= height
            bbox[2] /= width
            bbox[3] /= height
            if len([x for x in bbox if 0 <= x <= 1]) == 4:
              new_bboxes.append(bbox)
              new_cont_labels.append([float(x)/10 for x in row["continuous_labels"][index][1:-1].split(',')])
              new_labels.append([x for x in row["label"][index][1:-1].replace(" ", "").replace("'", "").split(',')])
        if len(new_bboxes)<MAX_BOX:
            box_to_add=MAX_BOX-len(new_bboxes)
            for i in range(box_to_add):
                new_bboxes.append([0.0,0.0,0.0,0.0])
                new_cont_labels.append([0.0,0.0,0.0])
                new_labels.append([])
        count += 1
        if count > limit[label]:
            break
        else:
            annotation[label].append({
                "path": row.name,
                "bbox": new_bboxes,
                "continuous_labels": new_cont_labels,
                "label": new_labels
            })

    print(f"Dataset {label} lenght post sampling: {len(annotation[label])}")
    #display(annotation[label][0])
    print()
  
  blacklist = [set(x["path"] for x in annotation["train"]).intersection(set(x["path"] for x in annotation["val"]))]
  blacklist += [set(x["path"] for x in annotation["test"]).intersection(set(x["path"] for x in annotation["val"]))]
  blacklist += [set(x["path"] for x in annotation["train"]).intersection(set(x["path"] for x in annotation["test"]))]
  for label in ["val", "train", "test"]:
    annotation[label] = [x for x in annotation[label] if x["path"] not in blacklist]

  print(f"\nRemoved {len(blacklist)} images for incorrect annotations")

  with open("Annotations.json", "w") as outfile:
    outfile.write(json.dumps(annotation, indent=4))

In [ ]:
# ----- Tensorflow Dataset V2 ------

BATCH_SIZE = 16
IMG_SIZE = (224, 224)
IMG_SHAPE = IMG_SIZE + (3,)

STR_LABEL = False

dataset = {
    "train": None,
    "val": None,
    "test": None,
}

def read_image_bbox(path, bbox, label):
    path = path.decode()
    image = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    image = cv2.resize(image, (IMG_SIZE[1], IMG_SIZE[0]))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image / 255.0

    image = image.astype(np.float32)
    norm_bbox = np.array(bbox, dtype=np.float32)
    categories = np.array(label, dtype=np.float32)

    return image, norm_bbox, categories

def parse(image, bbox, label):
    image,bbox,label = tf.numpy_function(read_image_bbox, [image,bbox,label], [tf.float32, tf.float32, tf.float32])
    image.set_shape([IMG_SIZE[0], IMG_SIZE[1], 3])
    bbox.set_shape([MAX_BOX,4])
    if STR_LABEL:
      label.set_shape([MAX_BOX,len(cat)])
    else:
      label.set_shape([MAX_BOX,3])
    return (image), (bbox, label)

for label in ["val", "train", "test"]:

  print(f"Annotation dataset {label} lenght post sampling: {len(annotation[label])}")
  

  def lambda_label(x):
    out = []
    for y in x["label"]:
       out.append(parse_categories(y))
    return out
      
  #lambda_label = lambda x: [parse_categories(y) y for y in x["label"]]

  if not STR_LABEL:
    lambda_label = lambda x: x["continuous_labels"]

  dataset[label] = tf.data.Dataset.from_tensor_slices((
        [x["path"] for x in annotation[label]],
        [x["bbox"] for x in annotation[label]],
        [lambda_label(x) for x in annotation[label]]
  ))

  dataset[label] = dataset[label].map(parse)

  for n, image in enumerate(dataset[label].take(1)):
    printImage(image[1][0].numpy(), image[1][1].numpy(), image[0], label_mode=STR_LABEL)

  dataset[label] = dataset[label]

In [ ]:
#  ----- Network Architecture ----- 

tf.keras.utils.set_random_seed(12345)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False
#base_model.summary()

inputs = tf.keras.Input(IMG_SHAPE)
#x = tf.keras.applications.mobilenet_v2.preprocess_input(inputs)
x = base_model(inputs, training=False)
#x = tf.keras.layers.Conv2D(256, kernel_size=1, padding="same")(x)
#x = tf.keras.layers.BatchNormalization()(x)
#x = tf.keras.layers.Activation("relu")(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
#x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
bbox = tf.keras.layers.Dense(MAX_BOX*4, activation="sigmoid")(x) 
bbox=tf.keras.layers.Reshape((MAX_BOX,4),name="bbox")(bbox)
labels = tf.keras.layers.Dense((MAX_BOX*3), activation="sigmoid")(x)
labels=tf.keras.layers.Reshape((MAX_BOX,3),name="label")(labels) 
if STR_LABEL:
    labels = tf.keras.layers.Dense((MAX_BOX*len(cat)), activation="sigmoid")(x) 
    labels=tf.keras.layers.Reshape((MAX_BOX,len(cat)),name="label")(labels)
model = tf.keras.Model(inputs=[inputs], outputs=[bbox,labels]) 
model.summary()

In [ ]:
# ----- Training for the top layers -----


initial_epochs = 10
base_learning_rate = 0.001

callbacks = [
    tf.keras.callbacks.ModelCheckpoint("freeze.h5", verbose=1, save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-8, verbose=1),
]

model.compile(optimizer=tf.keras.optimizers.legacy.Adam(base_learning_rate),
              #run_eagerly=True,
              loss={
                  "bbox": tf.keras.losses.MeanSquaredError(), #tfa.losses.GIoULoss()
                  "label": tf.keras.losses.CategoricalCrossentropy() if STR_LABEL  else tf.keras.losses.MeanSquaredError()})

model.summary()


history = model.fit(
    dataset["train"].batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE),
    epochs=initial_epochs,
    callbacks=callbacks,
    validation_data=dataset["val"].batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)

initial_epochs = len(history.history["loss"]) - 1 

In [ ]:
# ----- Plot Train/Val Loss -----

loss = [x for x in history.history['loss']]
val_loss = [x for x in history.history['val_loss']]

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Loss')
#plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# ----- Plot Random Prediction -----

def run_detector(detector, path, test=False, box=[], label=[]):
  image = cv2.imread(path, cv2.IMREAD_UNCHANGED)
  image = cv2.resize(image, (IMG_SIZE[1], IMG_SIZE[0]))
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = image / 255.0
  image = image.astype(np.float32)
  converted_img  = tf.image.convert_image_dtype(image, tf.float32)[tf.newaxis, ...]
  result = detector(converted_img)
  #print(result[0][0].numpy())
  if test:
    if label != []:
      result_labels = []
      for i in range(len(label)):
        if STR_LABEL:
          result_labels.append(parse_index_from_cat(result[1][0].numpy()[i]))
        else:
          #print(result[1][0].numpy()[i])
          result_labels.append([round(x,1) for x in result[1][0].numpy()[i]])
        #print(result[1][0].numpy()[i])
        #print(result_labels)
      printImage(result[0][0].numpy(), result_labels, path)
      printImage(box, label, path)
    else:
      printImage(result[0][0].numpy(), ["" for x in range(MAX_BOX)], path)
      printImage(box, ["" for x in range(MAX_BOX)], path)
  else:
    printImage(result[0][0].numpy(), result[1][0].numpy(), path)

model.load_weights("freeze.h5")

n = random.randint(0, len(annotation["val"]))
n = annotation["val"][n]
#result = model.predict(tf.reshape(image[0], (1, 224, 224, 3)))
#print(n["path"])
result=run_detector(model,n["path"], test=True, box=n["bbox"], label=n["label"] if STR_LABEL else n["continuous_labels"])

In [ ]:
# ----- Fine Tuning -----

base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

#model_fine = tf.keras.Model(inputs=[inputs], outputs=[bbox,labels])

callbacks = [
    tf.keras.callbacks.ModelCheckpoint("finetune.h5", verbose=1, save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-8, verbose=1),
]

model.compile(optimizer=tf.keras.optimizers.Adam(base_learning_rate/10), 
                   loss={
                        "bbox": tf.keras.losses.MeanSquaredError(), #tfa.losses.GIoULoss()
                        "label": tf.keras.losses.CategoricalCrossentropy() if STR_LABEL  else tf.keras.losses.MeanSquaredError()
                    })


fine_tune_epochs = int(initial_epochs/2)
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(dataset["train"].batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE),
                         epochs=total_epochs,
                         callbacks=callbacks,
                         initial_epoch=history.epoch[-1],
                         validation_data=dataset["val"].batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)

In [ ]:
# ----- Plot Train/Val Loss -----

loss += [x for x in history_fine.history['loss']]
val_loss += [x for x in history_fine.history['val_loss']]

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
#plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
model.load_weights("finetune.h5")

n = random.randint(0, len(annotation["val"]))
n = annotation["val"][n]
#result = model.predict(tf.reshape(image[0], (1, 224, 224, 3)))
#print(n["path"])
result=run_detector(model,n["path"], test=True, box=n["bbox"], label=n["label"] if STR_LABEL else n["continuous_labels"])

In [ ]:
# ----- Evaluation -----

model.load_weights("finetune.h5")

results = model.evaluate(dataset["test"].batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE))
print(model.metrics_names)
print(results)